In [6]:

# =============================================================================
# Income and agricultural inputs august 2018
# =============================================================================

import numpy as np
import pandas as pd
import os 
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
os.chdir('C:/Users/rodri/Dropbox/PhD thesis/python')
from data_functions_albert import remove_outliers, gini
os.chdir('C:/Users/rodri/Dropbox/Chiefs_Field_Aug_18/Data/')
percentiles = [0.05, 0.1, .25, .5, .75, 0.8, 0.9, 0.95, 0.99]
euro_MWK = 806.64
#Import data
data = pd.read_csv("data_SIEG_clean.csv")

#Get ids: once done, no need to redo it
'''
ids = pd.read_csv('hhids.csv')
data = pd.merge(data, ids, on='intervieweename', how='inner')
data.rename(columns={'HHID':'hhid'}, inplace=True)
data[['hhid','intervieweename']]

data.to_csv('data_SIEG_clean.csv', index=False)
data.to_stata('data_SIEG_clean.dta')
'''

pd.value_counts(data['manylabor'])
## 27 families have one member working. 1 family has 2 members.
pd.value_counts(data['manyjobs_1'])
## All members have 1 job.


# Rename some variables
data.rename(columns={'HHID':'hhid'}, inplace=True) 
data.rename(columns={'unitssoldpearlmillet2':'unitssoldpearlmilletout2'}, inplace=True) 
data.rename(columns={'unitssoldsoyabean2':'unitssoldsoyabeanout2'}, inplace=True)
data.rename(columns={'soldquantitygroundbeanin':'soldquantitygroundbeanin'}, inplace=True)


# check number of plots
data['total_plots'] = data['manyplot'].fillna(0) + data['rentinmany'].fillna(0)
sumplots = data[['manyplot','rentinmany','total_plots']].describe()
### All households own plots! 32 households rented-in a plot for the rainy season.
## Most households have 1 plot. only top 25% have 2 or more. Maximum of plots: 6.
data['cultivatelandrain'].describe()
# 96.5% of households cultivated land.

#Check transfers
data[['govtransf', 'remittances']].describe()
pd.value_counts(data['govtransf'])
# 26 households received gov transfers. High dispersion in the transfer value. from 700 to 30000 MWK
pd.value_counts(data['remittances'])
# 24 households received remittances. from 2000 to 100000 MWK.


## Check plot size and value =========================================
#units area plots
units_plot = pd.value_counts(data['unitsareaplot_1'])
# most households (261) report plots in acres.
# small futbol fields are around 1 acre

## replace outlier, Yussuf:



# hh aggregate variables
data['hh_area_plots'] = 0
data['hh_rentout_plots'] = 0
data['hh_value_plots'] = 0

N_plots = int(data[['total_plots']].max())

for i in range(1,N_plots+1):
    data['area_plot_acr_'+str(i)] = data['areaplot_'+str(i)]
    data.loc[data['unitsareaplot_'+str(i)]==3.0, 'area_plot_acr_'+str(i)] = data.loc[data['unitsareaplot_'+str(i)]==3.0, 'areaplot_'+str(i)]*2.47105

data.loc[data['hhid']==209, 'area_plot_acr_3'] = np.nan

for i in range(1,N_plots+1):
    data['hh_area_plots'] += data['areaplot_'+str(i)].fillna(0)
    data['hh_rentout_plots'] += data['rentoutplot_'+str(i)].fillna(0)
    data['hh_value_plots'] += data['valueplot_'+str(i)].fillna(0)
    

data['hh_p_acre_plots']  =  data['hh_value_plots'] / data['hh_area_plots'] 
data['hh_rent_per_acre']  =  data['hh_rentout_plots'] / data['hh_area_plots'] 
data['hh_ratio_value_rent'] = data['hh_value_plots'] / data['hh_rentout_plots']




## REINTERVIEW THOSE HOUSEHOLDS WITH EXTREME VALUES:
big_areas = data.loc[data['hh_area_plots']>5,['hhid','intervieweename','hh_area_plots','hh_rentout_plots','hh_value_plots']]
# 15 households reported areas above 5 acres. One household (yussuf ahmadu) owing reported 1002 acres. Wrong.
# Davie Manejee reported 19 acres.

### Yussuf Ahmadu reported well first and second plot but not 3th. Let's replace it above for nan value
data.loc[ data['hhid']==209,['area_plot_acr_1', 'area_plot_acr_2', 'area_plot_acr_3', 'hh_area_plots']] 
## still messing it 
data.loc[ data['hhid']==209,'hh_area_plots'] = data.loc[ data['hhid']==209,['area_plot_acr_1', 'area_plot_acr_2']].sum(axis=1)
# Summarize land rights ===============================================
sum_landrights = data[['rightsellplot', 'rightbequeathplot', 'chiefpreventsell', 'chiefpreventbequeat', 'landdispute' ]].describe()
## around 40% of households can sell/bequeth their plot. Around 25% households reported that 
# chief can prevent them to sel/bequeath land.

sum_solverdispute = pd.value_counts(data['whosolveddispute'])
## most land disputes solved by the chief!!

sum_expropiation = pd.value_counts(data['expropiation'])
## Expropiation techniques: Cultivate land, demarcate borders, report chief


sum_1plot = data[['area_plot_acr_1','rentoutplot_1','valueplot_1']].describe(percentiles=percentiles)
   
sum_2plot = data[['area_plot_acr_2','rentoutplot_2','valueplot_2']].describe(percentiles=percentiles)

sum_hhplots = data[['hh_area_plots','hh_rentout_plots','hh_value_plots', 'hh_ratio_value_rent', 'hh_p_acre_plots']].describe(percentiles=percentiles)
print('=========== SUMMARY HOUSEHOLDS LAND ============')
sum_hhplots

=========== SUMMARY HOUSEHOLDS LAND ============


,hh_area_plots,hh_rentout_plots,hh_value_plots,hh_ratio_value_rent,hh_p_acre_plots
count,269.000000,2.690000e+02,2.690000e+02,2.650000e+02,2.660000e+02
mean,2.333643,3.340919e+04,4.844386e+05,inf,1.723032e+05
std,2.031233,9.913411e+04,2.164316e+06,NaN,3.476268e+05
min,0.000000,0.000000e+00,0.000000e+00,1.111000e-04,0.000000e+00
5%,0.500000,5.000000e+03,2.000000e+04,1.741414e+00,1.666667e+04
10%,1.000000,6.000000e+03,2.999940e+04,2.440000e+00,2.000000e+04
25%,1.000000,1.000000e+04,7.000000e+04,4.000000e+00,4.233333e+04
50%,2.000000,1.500000e+04,1.650000e+05,8.333333e+00,9.571429e+04
75%,3.000000,3.000000e+04,3.300000e+05,1.500000e+01,1.500000e+05
80%,3.000000,3.500000e+04,4.000000e+05,1.875000e+01,2.000000e+05


In [7]:
#Convert agricultural outputs to kgs ====================================
# Import conversion rates
crop_unit = pd.read_csv("crop_conversions_kg.csv")
list_crops = ['maize', 'groundnut', 'groundbean', 'sweetpotatoe', 'fingermillet', 'sorghum', 'pearlmillet', 'soyabean', 'pigeonpeas', 'cotton', 'nkhwani', 'cassava',  'sugarcane',  'tomatoes',  'therereokra', 'tanaposi' ] 

#Generate empty variables
for crop in list_crops:
    data['total_kg_'+crop] = np.nan 
    data['sold_kg_'+crop] = np.nan 
    data['sold_insiders_kg_'+crop] = np.nan 
    data['sold_outsiders_kg_'+crop] = np.nan 
    data['sold_outside_kg_'+crop] = np.nan 
    data['store_kg_'+crop] = np.nan 
    data['lost_kg_'+crop] = np.nan
    data['total2_kg_'+crop] =np.nan
    data['sold_bigger_total_'+crop] = 0
    data['lost_bigger_total_'+crop] = 0
    data['store_bigger_total_'+crop] = 0
    data['soldloststore_bigger_total_'+crop] = 0
    data['p_'+crop] = np.nan
    data['y_'+crop] = 0
    data['y_agric'] = 0
    data[['unitstotal'+crop, 'unitssold'+crop, 'unitsstore'+crop,'unitslost'+crop]].replace(np.nan, 0, inplace=True)
  
       
# =============================================================================
# Main Loop: Conversion to kgs for all crops and questions    
# =============================================================================

data.replace(np.nan, 0, inplace=True)
for i in range(len(data)): 
    for crop in list_crops:
        data.iloc[i, data.columns.get_loc('total_kg_'+crop)] = data.iloc[i,data.columns.get_loc('totalamount'+crop)]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitstotal'+crop)]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop)]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop)]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_insiders_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop+'in')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop+'in')]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_outsiders_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop+'out')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop+'out')]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_outside_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop+'out2')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop+'out2')]),'conversionkg']  
        data.iloc[i, data.columns.get_loc('store_kg_'+crop)] = data.iloc[i,data.columns.get_loc('store'+crop+'quantity')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitsstore'+crop)]),'conversionkg']
        data.iloc[i, data.columns.get_loc('lost_kg_'+crop)] = data.iloc[i,data.columns.get_loc('lost'+crop+'quantity')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitslost'+crop)]),'conversionkg']


for crop in list_crops:
    data['total2_kg_'+crop] = data['sold_kg_'+crop].fillna(0) +data['store_kg_'+crop].fillna(0) +data['lost_kg_'+crop].fillna(0) 
   
#Summary total output kg:
sum_kg = (data[['total_kg_maize', 'total_kg_groundnut', 'total_kg_groundbean', 'total_kg_sweetpotatoe', 'total_kg_fingermillet', 'total_kg_sorghum', 'total_kg_pearlmillet', 'total_kg_soyabean', 'total_kg_pigeonpeas', 'total_kg_cotton', 'total_kg_nkhwani', 'total_kg_cassava',  'total_kg_sugarcane',  'total_kg_tomatoes',  'total_kg_therereokra', 'total_kg_tanaposi']].replace(0,np.nan)).describe()
# most people produce maize and groundnuts. Maize: mean of 438kg, sd 519 kg. Groundnut: mean 378kg, sd 940. min of 35 (both), max(maize) 3920kg, max(groundnut) 10500kg.
# No one produced tanaposi and sugarcane.
print('=========== SUMMARY CROP PRODUCTION (IN KG) RAINY SEASON ============')
sum_kg

C:\Users\rodri\Anaconda3\lib\site-packages\pandas\core\frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


=========== SUMMARY CROP PRODUCTION (IN KG) RAINY SEASON ============


,total_kg_maize,total_kg_groundnut,total_kg_groundbean,total_kg_sweetpotatoe,total_kg_fingermillet,total_kg_sorghum,total_kg_pearlmillet,total_kg_soyabean,total_kg_pigeonpeas,total_kg_cotton,total_kg_nkhwani,total_kg_cassava,total_kg_sugarcane,total_kg_tomatoes,total_kg_therereokra,total_kg_tanaposi
count,241.000000,152.000000,19.000000,53.000000,5.000000,2.0,4.000000,22.000000,116.000000,11.000000,20.000000,18.000000,0.0,2.0,9.000000,0.0
mean,438.029046,378.421053,619.473684,241.839623,320.000000,70.0,276.250000,157.045455,641.702586,4975.454545,160.125000,198.194444,NaN,210.0,367.222222,NaN
std,519.524377,940.466353,626.242975,342.630573,520.768663,0.0,164.082042,254.869991,1136.281435,4918.122332,308.342849,382.849039,NaN,0.0,506.785896,NaN
min,35.000000,35.000000,35.000000,17.500000,70.000000,70.0,105.000000,35.000000,17.500000,70.000000,17.500000,17.500000,NaN,210.0,35.000000,NaN
25%,210.000000,70.000000,52.500000,70.000000,70.000000,70.0,213.750000,43.750000,70.000000,335.000000,70.000000,70.000000,NaN,210.0,70.000000,NaN
50%,280.000000,210.000000,245.000000,140.000000,70.000000,70.0,250.000000,87.500000,140.000000,5000.000000,70.000000,70.000000,NaN,210.0,70.000000,NaN
75%,490.000000,350.000000,1250.000000,210.000000,140.000000,70.0,312.500000,140.000000,1250.000000,7550.000000,70.000000,70.000000,NaN,210.0,280.000000,NaN
max,3920.000000,10500.000000,1750.000000,1750.000000,1250.000000,70.0,500.000000,1250.000000,6500.000000,15000.000000,1400.000000,1250.000000,NaN,210.0,1250.000000,NaN


In [8]:

# Summary total sellings kg:
sum_sold_kg= (data[['sold_kg_maize', 'sold_kg_groundnut', 'sold_kg_groundbean', 'sold_kg_sweetpotatoe', 'sold_kg_fingermillet', 'sold_kg_sorghum', 'sold_kg_pearlmillet', 'sold_kg_soyabean', 'sold_kg_pigeonpeas', 'sold_kg_cotton', 'sold_kg_nkhwani', 'sold_kg_cassava',  'sold_kg_sugarcane',  'sold_kg_tomatoes',  'sold_kg_therereokra', 'sold_kg_tanaposi']].replace(0,np.nan)).describe()
# Few households sell. Only 20 sold maize, 65 groundnunt, 20 pigenonpeas and rest of them lower.
# Averages of sellings similar to total kg. min_maize_sold = 70. mean_maize_sold=370

#Summary sellings inside kg:
sum_sold_kg_inside = (data[['sold_insiders_kg_maize', 'sold_insiders_kg_groundnut', 'sold_insiders_kg_groundbean', 'sold_insiders_kg_sweetpotatoe', 'sold_insiders_kg_fingermillet', 'sold_insiders_kg_sorghum', 'sold_insiders_kg_pearlmillet', 'sold_insiders_kg_soyabean', 'sold_insiders_kg_pigeonpeas', 'sold_insiders_kg_cotton', 'sold_insiders_kg_nkhwani', 'sold_insiders_kg_cassava',  'sold_insiders_kg_sugarcane',  'sold_insiders_kg_tomatoes',  'sold_insiders_kg_therereokra', 'sold_insiders_kg_tanaposi']].replace(0,np.nan)).describe()
# Most sellings are within villagers.

#Summary sellings in village to outsiders:
sum_sold_outsiders_kg_in = (data[['sold_outsiders_kg_maize', 'sold_outsiders_kg_groundnut', 'sold_outsiders_kg_groundbean', 'sold_outsiders_kg_sweetpotatoe', 'sold_outsiders_kg_fingermillet', 'sold_outsiders_kg_sorghum', 'sold_outsiders_kg_pearlmillet', 'sold_outsiders_kg_soyabean', 'sold_outsiders_kg_pigeonpeas', 'sold_outsiders_kg_cotton', 'sold_outsiders_kg_nkhwani', 'sold_outsiders_kg_cassava',  'sold_outsiders_kg_sugarcane',  'sold_outsiders_kg_tomatoes',  'sold_outsiders_kg_therereokra', 'sold_outsiders_kg_tanaposi']].replace(0,np.nan)).describe()
#Groundnut high level of sellings in village to outsiders.

# Summary sellings outside village:
sum_sold_outside_kg_out = (data[['sold_outside_kg_maize', 'sold_outside_kg_groundnut', 'sold_outside_kg_groundbean', 'sold_outside_kg_sweetpotatoe', 'sold_outside_kg_fingermillet', 'sold_outside_kg_sorghum', 'sold_outside_kg_pearlmillet', 'sold_outside_kg_soyabean', 'sold_outside_kg_pigeonpeas', 'sold_outside_kg_cotton', 'sold_outside_kg_nkhwani', 'sold_outside_kg_cassava',  'sold_outside_kg_sugarcane',  'sold_outside_kg_tomatoes',  'sold_outside_kg_therereokra', 'sold_outside_kg_tanaposi']].replace(0,np.nan)).describe()
## Very low levels of sellings outside village (8hh in groundnut, and for the rest of crops mostly 0)
# Sum transportation costs
sum_transport_c = (data[['transcostmaizeout', 'transcostgroundnutout', 'transcostgroundbeanout', 'transcostsweetpotatoeout', 'transcostfingermilletout', 'transcostsorghumout', 'transcostpearlmilletout', 'transcostsoyabeanout', 'transcostpigeonpeasout', 'transcostcottonout', 'transcostnkhwaniout', 'transcostcassavaout',  'transcostsugarcaneout',  'transcosttomatoesout',  'transcosttherereokraout', 'transcosttanaposiout']].replace(0,np.nan)).describe()
#Low transport costs due to most sellings within village.


# =============================================================================
# Check quantity sold, store, and lost not larger than total
# =============================================================================
for crop in list_crops:
    data['sold_bigger_total_'+crop] = 1*(data['sold_kg_'+crop]> data['total_kg_'+crop]+5)
    data['lost_bigger_total_'+crop] = 1*(data['lost_kg_'+crop]> data['total_kg_'+crop]+5)
    data['store_bigger_total_'+crop] = 1*(data['store_kg_'+crop]> data['total_kg_'+crop]+5)
    data['soldloststore_bigger_total'+crop] = 1*(data['total2_kg_'+crop]>data['total_kg_'+crop]+5)
    
check_sold_bigger_total = data[['sold_bigger_total_maize', 'sold_bigger_total_groundnut', 'sold_bigger_total_groundbean', 'sold_bigger_total_sweetpotatoe', 'sold_bigger_total_fingermillet', 'sold_bigger_total_sorghum', 'sold_bigger_total_pearlmillet', 'sold_bigger_total_soyabean', 'sold_bigger_total_pigeonpeas', 'sold_bigger_total_cotton', 'sold_bigger_total_nkhwani', 'sold_bigger_total_cassava',  'sold_bigger_total_sugarcane',  'sold_bigger_total_tomatoes',  'sold_bigger_total_therereokra', 'sold_bigger_total_tanaposi']]


#Get the households that reported larger amounts than total:

list_hh_check_sell = []
list_hh_check_lost = []
list_hh_check_store = []
list_hh_check = []

for crop in list_crops:
    liers_sell = data.loc[data['sold_bigger_total_'+crop]==1, 'intervieweename']
    liers_store = data.loc[data['store_bigger_total_'+crop]==1, 'intervieweename']
    liers_lost = data.loc[data['lost_bigger_total_'+crop]==1, 'intervieweename']
    liers = data.loc[data['soldloststore_bigger_total_'+crop]==1, 'intervieweename']
    
    list_hh_check_sell.append(liers_sell)
    list_hh_check_store.append(liers_store)
    list_hh_check_lost.append(liers_lost)
    list_hh_check.append(liers)

hh_to_check_sell = pd.concat(list_hh_check_sell, axis=1)
hh_to_check_sell.columns = list_crops

hh_to_check_store = pd.concat(list_hh_check_store, axis=1)
hh_to_check_store.columns = list_crops

hh_to_check_lost = pd.concat(list_hh_check_lost, axis=1)
hh_to_check_lost.columns = list_crops

hh_to_check = pd.concat(list_hh_check, axis=1)
hh_to_check.columns = list_crops

### NO ONE REPORTED SELLINGS, LOSTS, STORING PLUS THE 3 TOGHETHER. GREAT!

# get prices per kg
for crop in list_crops:
    data['p_'+crop] = (data['soldvalue'+crop].replace(0,np.nan)).dropna()  / (data['sold_kg_'+crop].replace(0,np.nan)).dropna() 
    #DF = data[['soldvalue'+crop, 'sold_kg_'+crop]].dropna()

sum_prices = data[['p_maize', 'p_groundnut', 'p_groundbean', 'p_sweetpotatoe', 'p_fingermillet', 'p_sorghum', 'p_pearlmillet', 'p_soyabean', 'p_pigeonpeas', 'p_cotton', 'p_nkhwani', 'p_cassava',  'p_sugarcane',  'p_tomatoes',  'p_therereokra', 'p_tanaposi']].describe()
print('===============================================================')
print('Check: Distribution of prices')
print('===============================================================')  
sum_prices.dropna(axis=1)  
   

Check: Distribution of prices


,p_maize,p_groundnut,p_sweetpotatoe,p_fingermillet,p_pearlmillet,p_pigeonpeas,p_cotton
count,20.000000,65.000000,18.000000,2.000000,2.000000,18.000000,13.000000
mean,60.581587,78.394174,64.484921,64.285714,25035.714286,11169.365079,103.960244
std,26.888004,36.734817,41.023159,10.101525,35304.831432,36585.738945,285.154474
min,1.600000,2.857143,1.066667,57.142857,71.428571,1.600000,2.000000
25%,56.746032,71.428571,29.315476,60.714286,12553.571429,2.100000,5.000000
50%,71.428571,71.428571,65.357143,64.285714,25035.714286,32.142857,8.666667
75%,72.142857,85.714286,93.214286,67.857143,37517.857143,92.857143,42.857143
max,85.714286,257.142857,142.857143,71.428571,50000.000000,150000.000000,1047.619048


In [9]:
   
#Get monetary value:
data['y_agric'] = 0
for crop in list_crops:
    data['y_'+crop] =  np.nanmedian(data['p_'+crop])*data['total_kg_'+crop].fillna(0)
    data['y_agric'] += data['y_'+crop].fillna(0)

    
## Descriptive statistics
sum_y = data[['y_agric','y_maize', 'y_groundnut', 'y_pigeonpeas', 'y_sweetpotatoe', 'y_sorghum', 'y_pearlmillet']].describe(percentiles=percentiles)
print('===============================================================')
print('Check: Distribution of output')
print('===============================================================')  
print('NOTE: We cant recover monetary value for many crops since no one sold them. Need to use some other prices')
sum_y.dropna(axis=1)
    

Check: Distribution of output
NOTE: We cant recover monetary value for many crops since no one sold them. Need to use some other prices


C:\Users\rodri\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4033: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


,y_agric,y_maize,y_groundnut,y_pigeonpeas,y_sweetpotatoe,y_pearlmillet
count,2.690000e+02,269.000000,269.000000,269.000000,269.000000,2.690000e+02
mean,1.605854e+05,28031.067446,15273.499734,8894.549920,3114.182820,1.028419e+05
std,9.450098e+05,36398.164520,52180.477588,26021.956769,11703.444779,9.445649e+05
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00
5%,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00
10%,5.000000e+03,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,1.875000e+04,10000.000000,0.000000,0.000000,0.000000,0.000000e+00
50%,3.918750e+04,20000.000000,5000.000000,0.000000,0.000000,0.000000e+00
75%,6.767857e+04,32142.857143,15000.000000,3375.000000,0.000000,0.000000e+00
80%,7.600714e+04,35000.000000,20000.000000,4500.000000,0.000000,0.000000e+00


In [10]:
#%% INPUTS

# =============================================================================
# Capital and livestock
# =============================================================================

#livestock
data['hhlivestock'] =0
for i in range(1,16):
    data['hhlivestock'] += (data['selllivstck_'+str(i)].replace(9999,np.nan)).fillna(0)

# Farm Equipment
data['hhfarmequip'] =0
for i in range(1,15):
    data['hhfarmequip'] += (data['sellfrmeqp_'+str(i)].replace(9999,np.nan)).fillna(0)

# Farm Structure
data['hhfarmstruct'] =0
for i in range(1,10):
    data['hhfarmstruct'] += (data['sellfrmstrc_'+str(i)].replace(9999,np.nan)).fillna(0)

# Farm Assets
data['hhfarmasset'] =0
for i in range(1,12):
    data['hhfarmasset'] += (data['sellhhasset_'+str(i)].replace(9999,np.nan)).fillna(0)
    

## farming capital   
data['k_farm'] = data['hhfarmequip'].fillna(0)+data['hhfarmstruct'].fillna(0)+data['hhfarmasset'].fillna(0)

## farming capital as in 2019 wave
data['k_farm_2'] = data['hhfarmequip'].fillna(0)


sum_capital = data[['k_farm','k_farm_2']].describe(percentiles=percentiles)
print('======= Summary Statistics Capital =======')
print('k_farm_2 is the k_farm in the wave19: i.e. only only farming equipment')
sum_capital

======= Summary Statistics Capital =======
k_farm_2 is the k_farm in the wave19: i.e. only only farming equipment


,k_farm,k_farm_2
count,2.690000e+02,269.000000
mean,5.384481e+04,8034.572491
std,1.182482e+05,15693.870426
min,0.000000e+00,0.000000
5%,1.000000e+03,500.000000
10%,1.940000e+03,1000.000000
25%,5.000000e+03,2300.000000
50%,1.980000e+04,4500.000000
75%,5.480000e+04,8300.000000
80%,6.702000e+04,10580.000000


In [11]:
#%% =============================================================================
# intermediates
# =============================================================================
# obtain value non-bought fertilizer. Use median price
data['p_fert']  = data['buyfertilizierpay'].divide(data['buyfertilizerkg'])
data['value_fertilizer'] = np.nanmedian(data['p_fert'])*data['fertilizerkg']

data['interm'] = (data['spendseeds'].fillna(0) +data['value_fertilizer'].fillna(0) +data['spendpesticides'].fillna(0)).replace(np.nan,0)

sum_interm = data[['interm','value_fertilizer','fertilizerkg', 'spendseeds', 'spendpesticides']].describe()
print('============= Summary of intermediate inputs (in MWK, except kg) ==============')
sum_interm


============= Summary of intermediate inputs (in MWK, except kg) ==============


,interm,value_fertilizer,fertilizerkg,spendseeds,spendpesticides
count,269.000000,269.000000,269.000000,269.000000,269.000000
mean,26458.847584,23779.182156,59.447955,1982.527881,697.137546
std,28124.234540,26413.681116,66.034203,3584.071770,2213.446607
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8000.000000,4000.000000,10.000000,0.000000,0.000000
50%,20000.000000,20000.000000,50.000000,600.000000,0.000000
75%,40000.000000,40000.000000,100.000000,2600.000000,0.000000
max,173000.000000,160000.000000,400.000000,39000.000000,18000.000000


In [12]:
#%% =============================================================================
# labor
# =============================================================================
data['labor_N'] = data['manyhhlaborplot'].fillna(0) +data['hireplotmany'].fillna(0)

#### Loop for hh labor input 
data['hh_labor_days'] = 0
data['hh_labor_hours'] = 0

for member in range(1,int(np.max(data['manyhhlaborplot'])+1)):           
    data['member_'+str(member)+'_labor_days'] =  (data['monthshhplot_'+str(member)].multiply(data['weekshhplot_'+str(member)],axis=0, fill_value=0)).multiply(data['dayshhplot_'+str(member)],axis=0, fill_value=0)  
    data['member_'+str(member)+'_labor_hours'] = data['member_'+str(member)+'_labor_days'].multiply(data['hourshhplot_'+str(member)],axis=0, fill_value=0)

for member in range(1,int(np.max(data['manyhhlaborplot'])+1)):     
    data['hh_labor_days'] += data['member_'+str(member)+'_labor_days'].fillna(0)
    data['hh_labor_hours'] += data['member_'+str(member)+'_labor_hours'].fillna(0)
    
    
    
#### Loop for hired labor input 
for hired in range(1,int(np.max(data['hireplotmany'])+1)):    
    data['months_hired_'+str(hired)] = np.nan           
    data['weeks_hired_'+str(hired)] = np.nan             
    data['days_hired_'+str(hired)] = np.nan 
    data['hours_hired_'+str(hired)] = np.nan 
    

data['hired_labor_days'] = 0
data['hired_labor_hours'] = 0

for hired in range(1,int(np.max(data['hireplotmany'])+1)):           
    data['hired_'+str(hired)+'_labor_days'] =  (data['hireplotmonths_'+str(hired)].multiply(data['hireplotweeks_'+str(hired)],axis=0, fill_value=0)).multiply(data['hireplotdays_'+str(hired)],axis=0, fill_value=0)  
    data['hired_'+str(hired)+'_labor_hours'] = data['hired_'+str(hired)+'_labor_days'].multiply(data['hireplothours_'+str(hired)],axis=0, fill_value=0)

for hired in range(1,int(np.max(data['hireplotmany'])+1)):     
    data['hired_labor_days'] += data['hired_'+str(hired)+'_labor_days'].fillna(0)
    data['hired_labor_hours'] += data['hired_'+str(hired)+'_labor_hours'].fillna(0)
    
    

data['hhlabor_N'] = data['manyhhlaborplot']
data['hired_N'] = data['hireplotmany']
data['labor_N'] = data['hhlabor_N'].fillna(0) +data['hired_N'].fillna(0)

data['labor_h'] = (data['hh_labor_hours'].fillna(0)+data['hired_labor_hours'].fillna(0)).replace(0,np.nan)
    
sum_labor = data[['labor_N', 'hhlabor_N', 'hired_N', 'labor_h', 'hh_labor_hours', 'hired_labor_hours']].describe(percentiles=percentiles)

print('=========== Agricultural labor input  =============')
print('Where _N denotes labor supply in number of persones, _h, denotes it in hours')
sum_labor

=========== Agricultural labor input  =============
Where _N denotes labor supply in number of persones, _h, denotes it in hours


,labor_N,hhlabor_N,hired_N,labor_h,hh_labor_hours,hired_labor_hours
count,269.000000,269.000000,269.000000,2.570000e+02,2.690000e+02,269.000000
mean,2.516729,2.509294,0.007435,9.328651e+06,8.912502e+06,0.490706
std,1.441877,1.439355,0.121942,1.496785e+08,1.463018e+08,8.048182
min,0.000000,0.000000,0.000000,-9.599040e+05,-9.599040e+05,0.000000
5%,1.000000,1.000000,0.000000,1.644000e+02,0.000000e+00,0.000000
10%,1.000000,1.000000,0.000000,2.304000e+02,1.680000e+02,0.000000
25%,2.000000,2.000000,0.000000,4.080000e+02,3.760000e+02,0.000000
50%,2.000000,2.000000,0.000000,6.960000e+02,6.400000e+02,0.000000
75%,3.000000,3.000000,0.000000,1.152000e+03,1.152000e+03,0.000000
80%,4.000000,4.000000,0.000000,1.305600e+03,1.296000e+03,0.000000


In [13]:
#%% =============================================================================
# LABOR INCOME: SALARY LABOR (last 3 months) ----------------------
# =============================================================================

## propotion households getting sallary work:
data['laborhh'].replace(2,0, inplace=True)
pd.value_counts(data['laborhh'])/len(data)

pd.value_counts(data['manylabor']) ## only 1 hh had 2 people working in a salary job
pd.value_counts(data['manyjobs_1']) #everyone had only 1 job

data['wlabor'] = 0
pd.value_counts(data['wageperiodjob_1_1'])
## labor supply in hours
data['wlabor_supply'] = 0
data['wlabor_inc'] = 0

for i in [1,2]:    
    data['wlabor_supply'] += ((data['weekslabor_'+str(i)+'_1'].multiply(data['dayslabor_'+str(i)+'_1'],axis=0, fill_value=0)).multiply(data['hourslabor_'+str(i)+'_1'],axis=0, fill_value=0)).fillna(0)  
    data.loc[data['wageperiodjob_'+str(i)+'_1']==1,'wlabor_inc'] += data.loc[data['wageperiodjob_'+str(i)+'_1']==1, 'wagejob_'+str(i)+'_1'].fillna(0)*3
    data.loc[data['wageperiodjob_'+str(i)+'_1']==2,'wlabor_inc'] += data.loc[data['wageperiodjob_'+str(i)+'_1']==2, 'wagejob_'+str(i)+'_1'].multiply(data['dayslabor_'+str(i)+'_1'],axis=0, fill_value=0).fillna(0)*3
    data.loc[data['wageperiodjob_'+str(i)+'_1']==3,'wlabor_inc'] += (data.loc[data['wageperiodjob_'+str(i)+'_1']==3, 'wagejob_'+str(i)+'_1'].multiply(data['weekslabor_'+str(i)+'_1'], axis=0, fill_value=0)).multiply(data['dayslabor_'+str(i)+'_1'],axis=0, fill_value=0).fillna(0)*3

## to rainy season:
data['wlabor_inc'] = data['wlabor_inc']*(7/3)

data['wlabor_inc_euro'] = data['wlabor_inc']/euro_MWK


## LABOR INCOME: GANYU (last 7 days) -----------------------------------
data['ganyuhh'].replace(2,0, inplace=True)
pd.value_counts(data['ganyuhh'])/len(data) ##40% ganyu in last 7 days

pd.value_counts(data['manyganyu'])

## GANYU AGRIC ---------
pd.value_counts(data['ganyuagric_1'])
pd.value_counts(data['wageperiodganyuagric_1'])

data['ganyuagri_supply'] = 0
data['ganyuagri_inc'] = 0
data['ganyuagri_inc2'] = 0
for i in [1,2]:    
    data['ganyuagri_supply'] += ((data['weeksganyuagric_'+str(i)].multiply(data['daysganyuagric_'+str(i)],axis=0, fill_value=0)).multiply(data['hoursganyuagric_'+str(i)],axis=0, fill_value=0)).fillna(0)  
    data.loc[data['wageperiodganyuagric_'+str(i)]==1,'ganyuagri_inc'] += data.loc[data['wageperiodganyuagric_'+str(i)]==1, 'wageganyuagric_'+str(i)].fillna(0)
    data.loc[data['wageperiodganyuagric_'+str(i)]==2,'ganyuagri_inc'] += data.loc[data['wageperiodganyuagric_'+str(i)]==2, 'wageganyuagric_'+str(i)].multiply(data['daysganyuagric_'+str(i)],axis=0, fill_value=0).fillna(0)
    data.loc[data['wageperiodganyuagric_'+str(i)]==3,'ganyuagri_inc'] += (data.loc[data['wageperiodganyuagric_'+str(i)]==3, 'wageganyuagric_'+str(i)].multiply(data['weeksganyuagric_'+str(i)], axis=0, fill_value=0)).multiply(data['daysganyuagric_'+str(i)],axis=0, fill_value=0).fillna(0)
    data['ganyuagri_inc2']  += data['wageganyuagric2_'+str(i)]*data['manyganyuagric_'+str(i)]


## GANYU NO-AGRIC -----
pd.value_counts(data['ganyuother_1'])
pd.value_counts(data['ganyuotheractivities_1'])
pd.value_counts(data['wageperiodganyuother_1'])    

data['ganyuother_supply'] = 0
data['ganyuother_inc'] = 0
data['ganyuother_inc2'] = 0
for i in [1,2]:    
    data['ganyuother_supply'] += ((data['weeksganyuother_'+str(i)].multiply(data['daysganyuother_'+str(i)],axis=0, fill_value=0)).multiply(data['hoursganyuother_'+str(i)],axis=0, fill_value=0)).fillna(0)  
    data.loc[data['wageperiodganyuother_'+str(i)]==1,'ganyuother_inc'] += data.loc[data['wageperiodganyuother_'+str(i)]==1, 'wageganyuother_'+str(i)].fillna(0)
    data.loc[data['wageperiodganyuother_'+str(i)]==2,'ganyuother_inc'] += data.loc[data['wageperiodganyuother_'+str(i)]==2, 'wageganyuother_'+str(i)].multiply(data['daysganyuother_'+str(i)],axis=0, fill_value=0).fillna(0)
    data.loc[data['wageperiodganyuother_'+str(i)]==3,'ganyuother_inc'] += (data.loc[data['wageperiodganyuother_'+str(i)]==3, 'wageganyuother_'+str(i)].multiply(data['weeksganyuother_'+str(i)], axis=0, fill_value=0)).multiply(data['daysganyuother_'+str(i)],axis=0, fill_value=0).fillna(0)
    data.loc[data['wageperiodganyuother_'+str(i)]==4,'ganyuother_inc'] += ((data.loc[data['wageperiodganyuother_'+str(i)]==3, 'wageganyuother_'+str(i)].multiply(data['weeksganyuother_'+str(i)], axis=0, fill_value=0)).multiply(data['daysganyuother_'+str(i)],axis=0, fill_value=0)).multiply(data['hoursganyuother_'+str(i)],axis=0, fill_value=0).fillna(0)   
    data['ganyuother_inc2']  += data['wageganyuother2_'+str(i)]*data['manyganyuother_'+str(i)]


### looking at the data there are outliers in ganyuother. One hh was earning 145,000 EUR.
data['ganyuother_inc'] = remove_outliers(data[['ganyuother_inc']], lq=0, hq=0.99)
data['ganyu_inc'] = data['ganyuagri_inc'].fillna(0) + data['ganyuother_inc'].fillna(0)
data['ganyu_inc2'] = data['ganyuagri_inc2'].fillna(0) + data['ganyuother_inc2'].fillna(0)
data['ganyu_supply'] = data['ganyuagri_supply'].fillna(0) + data['ganyuother_supply'].fillna(0)
euro_MWK = 806.64



sum_ganyu = ((data[['ganyu_inc', 'ganyuagri_inc', 'ganyuother_inc', 'ganyu_inc2', 'ganyuagri_inc2', 'ganyuother_inc2',]]/euro_MWK).replace(0, np.nan)).describe()
sum_ganyu2 = (data[[ 'ganyu_supply', 'ganyuagri_supply','ganyuother_supply']].replace(0, np.nan)).describe()

## to rainy season:
data['ganyu_inc'] = data['ganyu_inc']*(7)
data['ganyu_inc_euro'] = data['ganyu_inc']/euro_MWK
sum_labinc = (data[['wlabor_inc_euro', 'wlabor_supply', 'ganyu_inc_euro', 'ganyu_supply']].replace(0, np.nan)).describe()


print('===== Summary Salary and Ganyu Labor (at rainy season level, 7 months) in EUR')
sum_labinc


===== Summary Salary and Ganyu Labor (at rainy season level, 7 months) in EUR


C:\Users\rodri\Dropbox\PhD thesis\python\data_functions_albert.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["houtliers_"+serie] = df[serie].quantile(hq)
C:\Users\rodri\Dropbox\PhD thesis\python\data_functions_albert.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df[serie]>df["houtliers_"+serie]] = np.nan
C:\Users\rodri\Anaconda3\lib\site-packages\pandas\core\frame.py:3503: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

,wlabor_inc_euro,wlabor_supply,ganyu_inc_euro,ganyu_supply
count,28.000000,28.000000,95.000000,102.000000
mean,685.497868,93.821429,711.702840,9612.274510
std,614.356255,143.118199,659.720826,96064.303982
min,121.491620,7.000000,4.338986,2.000000
25%,128.867896,33.000000,149.695031,22.500000
50%,728.949717,36.000000,485.966478,36.000000
75%,749.776852,54.000000,1249.628087,74.250000
max,2249.330556,588.000000,3037.290489,970299.000000


In [14]:
### BUSINESS INCOME (last month) -------------------------------------------

pd.value_counts(data['businessmany'])
pd.value_counts(data['numberbusiness_1'])

type_business = pd.value_counts(data['whatbusiness_1_1'])
data['business_type'] = data['whatbusiness_1_1']

## No hh has 2 business. No need for loop

data['business_revenue'] = data['salesbusiness_1_1']
data['business_costs'] = data['laborbusiness_1_1'] + data['businesscosts_1_1']
data['business_profits'] = data['business_revenue'] - data['business_costs']

business_data = data.loc[data['businessmany']==1, ['hhid', 'business_type', 'business_revenue', 'business_costs', 'business_profits']]
business_data[['business_revenue', 'business_costs', 'business_profits']] = business_data[['business_revenue', 'business_costs', 'business_profits']]/euro_MWK

### to rainy season: 
data[['business_revenue', 'business_costs', 'business_profits']] = data[['business_revenue', 'business_costs', 'business_profits']]*7

sum_business = data[['business_revenue','business_costs','business_profits']].describe(percentiles=percentiles)
print('============== SUMMARY OF NON-AGRIC BUSINESS IN THE VILLAGE (IN MWK, RAINY SEASON) =================')
sum_business

============== SUMMARY OF NON-AGRIC BUSINESS IN THE VILLAGE (IN MWK, RAINY SEASON) =================


,business_revenue,business_costs,business_profits
count,2.690000e+02,2.690000e+02,269.000000
mean,3.289217e+04,1.745316e+04,15439.007435
std,1.376094e+05,1.011977e+05,68175.397101
min,0.000000e+00,0.000000e+00,-140000.000000
5%,0.000000e+00,0.000000e+00,0.000000
10%,0.000000e+00,0.000000e+00,0.000000
25%,0.000000e+00,0.000000e+00,0.000000
50%,0.000000e+00,0.000000e+00,0.000000
75%,0.000000e+00,0.000000e+00,0.000000
80%,1.400000e+03,0.000000e+00,0.000000


In [15]:
#%%
### OTHER SOURCES OF INCOME (3 months) --------------------------------------------------

data['gov_trans'] = data['govtransf']
data['other_inc'] = data[[ 'gov_trans', 'remittances']].sum(axis=1)
sum_other = (data[[ 'gov_trans', 'remittances']].replace(0,np.nan)/euro_MWK).describe()

### to rainy season: 
data[['other_inc','gov_trans','remittances']] = data[['other_inc','gov_trans','remittances']]*(7/3)

print('============== OTHER SOURCES INCOME (RAINY SEASON) =================')
print('values in EUROS')
sum_other

============== OTHER SOURCES INCOME (RAINY SEASON) =================
values in EUROS


,gov_trans,remittances
count,26.000000,24.000000
mean,12.859611,36.313184
std,9.336562,35.733441
min,0.867797,2.479421
25%,5.733661,6.198552
50%,11.715263,27.893484
75%,14.876525,51.138054
max,37.191312,123.971040


In [16]:
#%% AGGREGATE INCOME ------------------------------------------------------------
data['inctotal'] = data[['y_agric' ,'wlabor_inc', 'ganyu_inc', 'business_profits', 'other_inc']].sum(axis=1)
income = data[['hhid','inctotal','y_agric','y_maize', 'y_groundnut', 'y_pigeonpeas', 'wlabor_inc', 'ganyu_inc', 'business_profits', 'other_inc']].replace(0,np.nan)

sum_inc = (income.loc[:, income.columns != 'hhid']/euro_MWK).describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

var_list = ['inctotal','y_agric','y_maize', 'y_groundnut', 'y_pigeonpeas', 'wlabor_inc', 'ganyu_inc', 'business_profits', 'other_inc']
gini_stat= np.empty((1, len(var_list)))

for i,state in enumerate(var_list):
    gini_stat[:,i] = gini(income[state].dropna().values)

data_gini = pd.DataFrame(gini_stat, columns=var_list)    
data_gini.reset_index(inplace=True)
data_gini['index'] = 'gini'
sum_inc.reset_index(inplace=True)
sum_inc = sum_inc.append(data_gini, ignore_index=True)

print('======== SUMMARY AGGREGATE INCOME AT RAINY SEASON (MWK) =========')
sum_inc

======== SUMMARY AGGREGATE INCOME AT RAINY SEASON (MWK) =========


,index,inctotal,y_agric,y_maize,y_groundnut,y_pigeonpeas,wlabor_inc,ganyu_inc,business_profits,other_inc
0,count,260.000000,249.000000,241.000000,152.000000,116.000000,28.000000,95.000000,50.000000,45.000000
1,mean,570.463075,215.069754,38.787797,33.509465,25.570458,685.497868,711.702840,102.972652,62.526401
2,std,1286.795676,1216.443692,46.004270,83.278994,45.278354,614.356255,659.720826,173.955596,67.925342
3,min,-105.375384,1.859566,3.099276,3.099276,0.697337,121.491620,4.338986,-173.559457,5.785315
4,10%,18.595656,15.186452,9.297828,6.198552,2.092011,123.921452,61.440048,-9.111871,12.149162
5,25%,55.224671,27.893484,18.595656,6.198552,2.789348,128.867896,149.695031,14.101706,20.248603
6,50%,165.780274,52.377765,24.794208,18.595656,5.578697,728.949717,485.966478,49.898344,34.711891
7,75%,776.815822,86.779728,43.389864,30.992760,49.809793,749.776852,1249.628087,130.169592,86.779728
8,90%,1499.472680,183.103007,74.382624,49.588416,49.809793,1277.744719,1537.042547,263.810374,144.054349
9,max,15550.838753,15550.838753,347.118913,929.782803,259.010924,2249.330556,3037.290489,789.695527,300.836391


In [17]:
#%% ===========================================================================
#   Asset wealth
# =============================================================================

data['housing'] = data['selldwell']

data['hh_assets'] = 0
for i in range(1,12):
    data['hh_assets'] += data['sellhhasset_'+str(i)]


sum_assets = data[['housing','hh_assets']].describe(percentiles=percentiles)

print('Summary statistics Asset holdings. (MWK)')
print('Note that we didnt ask this questions in wave19')
sum_assets

Summary statistics Asset holdings. (MWK)
Note that we didnt ask this questions in wave19


,housing,hh_assets
count,2.690000e+02,2.690000e+02
mean,6.121301e+05,4.114184e+04
std,2.436704e+06,1.101847e+05
min,-9.999000e+03,0.000000e+00
5%,1.000000e+04,0.000000e+00
10%,2.500000e+04,0.000000e+00
25%,5.000000e+04,0.000000e+00
50%,1.500000e+05,1.000000e+04
75%,3.500000e+05,4.500000e+04
80%,5.000000e+05,5.000000e+04


In [19]:
#%% Obtain short data

  # Net agricultural output
data['y_net'] = (data['y_agric'].fillna(0) -data['interm'].fillna(0)).replace(np.nan,0)  
data['wave'] = 2018  



data_short =  data[['hhid', 'wave','intervieweename','householdsize', 'denominationhead', 'educationlevel', 'educationlevelspouse',
                    'villageheadfamily', 'relationshiptochief','rightsellplot', 'rightbequeathplot', 'chiefpreventsell', 'chiefpreventbequeat', 
                    'inctotal', 'y_net','y_agric','y_maize', 'y_groundnut', 'y_pigeonpeas', 'total_kg_maize', 'total_kg_groundnut',  'total_kg_pigeonpeas',
                    'wlabor_inc', 'ganyu_inc', 'ganyuagri_inc', 'ganyuother_inc', 'business_profits', 'other_inc', 'gov_trans', 'remittances',
                    'hh_area_plots', 'hh_rent_per_acre', 'hh_value_plots', 'hh_rentout_plots','labor_N','hhlabor_N','hired_N', 'labor_h', 'hh_labor_hours', 'hired_labor_hours',
                    'interm', 'value_fertilizer','fertilizerkg', 'k_farm', 'hhlivestock', 'housing', 'hh_assets']]

data_weird = data_short.loc[data_short['y_net']<0, ['y_net','y_agric','y_maize', 'total_kg_maize' ,'hh_area_plots','interm','value_fertilizer', 'fertilizerkg']]
data_weird.to_csv('neg_netoutput_18.csv')

data_short.to_csv('income_wealth_18.csv', index=False)
data_short.to_stata('C:/Users/rodri/Dropbox/CrossSecSSA - Village Misallocation/data/income_18_short.dta')


### y_net is agricultucal net income (minus intermediates). (MWK)
### y_agric is gross agricultural income (MWK)   
### Labor variables: N denotes unit is number of persons. labor_h, denotes total labor input (hh+hired) in hours.
### Shock variables: whether households reported the shock or not.
### I trim the variables: 'interm','labor_h', 'k_farm', 'hh_area_plots' at the 1% both tails to remove outliers. For example, in land area the 99% was around 20 acres. The maximum more than 100 acres...

